In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
from timeit import default_timer as timer
from _collections import OrderedDict
import shutil
from pprint import pprint
start = timer()
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)
from utilities.file_location import FileLocationManager
from utilities.sqlcontroller import SqlController
from utilities.utilities_cvat_neuroglancer import get_structure_number, NumpyToNeuroglancer, get_segment_properties
from utilities.utilities_affine import align_point_sets, load_atlas_data, estimate_structure_centers


Connecting dklab@192.168.1.12:3306


In [2]:
animal = 'DK52'
fileLocationManager = FileLocationManager(animal)
OUTPUT_DIR = os.path.join(fileLocationManager.neuroglancer_data, 'atlas')
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

atlas_data = load_atlas_data()
source_centers = estimate_structure_centers(atlas_data, animal)
# DK52 structure centers
output_centers = {
    '12N': [46488, 18778, 242],
    '5N_L': [38990, 20019, 172],
    '5N_R': [39184, 19027, 315],
    '7N_L': [42425, 23190, 166],
    '7N_R': [42286, 22901, 291]
}
structures = sorted(output_centers.keys())
source_point_set = np.array([source_centers[s][0] for s in structures]).T
# Unify to 1 mum scale in all axes
source_scale = np.diagflat([10, 10, 20])
source_point_set = source_scale @ source_point_set
output_point_set = np.array([output_centers[s] for s in structures]).T

# Unify to 1 mum scale in all axes
output_scale = np.diagflat([0.325, 0.325, 20])
output_point_set = output_scale @ output_point_set

r_auto, t_auto = align_point_sets(source_point_set, output_point_set)
t_auto_out = (np.linalg.inv(output_scale) @ t_auto).T


resolution: 0.325
width: 65000
height: 36000
box center: [1056.25  585.    242.5 ]


In [3]:
sql_controller = SqlController(animal)
thumbnail_dir = os.path.join(fileLocationManager.prep, 'CH1', 'thumbnail_aligned')

print(f'resolution: {sql_controller.scan_run.resolution}')
print(f'width: {sql_controller.scan_run.width}')
print(f'height: {sql_controller.scan_run.height}')
box_w = sql_controller.scan_run.width * sql_controller.scan_run.resolution / 10  # 10 mum scale
box_h = sql_controller.scan_run.height * sql_controller.scan_run.resolution / 10  # 10 mum scale
box_z = len(os.listdir(thumbnail_dir))  # 20 mum scale
box_center = np.array([box_w, box_h, box_z]) / 2

atlasV7_volume = np.zeros((int(box_h), int(box_w), int(box_z)), dtype=np.uint8)
print('Shape of atlas volume', atlasV7_volume.shape)


resolution: 0.325
width: 65000
height: 36000
Shape of atlas volume (1170, 2112, 485)


In [35]:
pp = list(source_centers.values())
x = np.array([(p[0][0],p[0][1],p[0][2]) for p in pp])

In [40]:
#x_out = inv(output_scale) @ x
#x can be replaced by output_point_set_expected_auto or t_auto.

x1 = r_auto @ x.T + t_auto
x2 = np.linalg.inv(output_scale) @ x1 
x2.T

array([[20356.12993158, 15771.88554679,    94.71747143],
       [20357.14814198, 15766.17473249,    96.48961898],
       [20336.18714094, 15779.2426259 ,    95.56960184],
       [19568.09862102, 15636.4310483 ,    95.20048783],
       [19568.56827266, 15633.79692347,    96.01789453],
       [19586.82998816, 15623.05165524,    94.84644026],
       [19587.68903452, 15618.23354081,    96.34157025],
       [19806.89767028, 15831.8836056 ,    91.55569064],
       [19811.69290405, 15804.98868342,    99.90157059],
       [19864.69256106, 15763.32914781,    94.55928973],
       [19865.96912444, 15756.16931483,    96.78108856],
       [20033.64104165, 15979.78700259,    92.29048936],
       [20037.69908056, 15957.02676899,    99.35331617],
       [19937.65943248, 15896.60203294,    92.61759421],
       [19941.27822106, 15876.305413  ,    98.91592637],
       [20250.14909259, 15687.89529991,    95.61164232],
       [20134.31558027, 15872.12821025,    91.93610757],
       [20138.65721967, 15847.7

In [22]:
arr = np.array(list(source_centers.values()))
arr = arr[0][0]
print(arr.shape)
x = r_auto @ arr + t_auto.T
xout = np.linalg.inv(output_scale) @ x
xout.T

(3,)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 3)

In [6]:
for structure, (source_point, volume) in sorted(source_centers.items()):
    print(str(structure).ljust(6),end=": ")
    results = r_auto @ source_point + t_auto.T
    results = np.linalg.inv(output_scale) @ results.T
    print(results)
    continue
    x = results[0][0]
    y = results[0][1]
    z = results[0][1]
    print(x,y,z)


10N_L : [[20356.12993158]
 [15771.88554679]
 [   94.71747143]]
10N_R : [[20357.14814198]
 [15766.17473249]
 [   96.48961898]]
12N   : [[20336.18714094]
 [15779.2426259 ]
 [   95.56960184]]
3N_L  : [[19568.09862102]
 [15636.4310483 ]
 [   95.20048783]]
3N_R  : [[19568.56827266]
 [15633.79692347]
 [   96.01789453]]
4N_L  : [[19586.82998816]
 [15623.05165524]
 [   94.84644026]]
4N_R  : [[19587.68903452]
 [15618.23354081]
 [   96.34157025]]
5N_L  : [[19806.89767028]
 [15831.8836056 ]
 [   91.55569064]]
5N_R  : [[19811.69290405]
 [15804.98868342]
 [   99.90157059]]
6N_L  : [[19864.69256106]
 [15763.32914781]
 [   94.55928973]]
6N_R  : [[19865.96912444]
 [15756.16931483]
 [   96.78108856]]
7N_L  : [[20033.64104165]
 [15979.78700259]
 [   92.29048936]]
7N_R  : [[20037.69908056]
 [15957.02676899]
 [   99.35331617]]
7n_L  : [[19937.65943248]
 [15896.60203294]
 [   92.61759421]]
7n_R  : [[19941.27822106]
 [15876.305413  ]
 [   98.91592637]]
AP    : [[20250.14909259]
 [15687.89529991]
 [   95.611

In [41]:
z_length = 486
volume_shape = 337
SC_start = 127
SC_end = 311
midz = 238
z_origin = -185
midz - midz/2

119.0

In [44]:
print('end - start=',SC_end - SC_start)
print('half of volume=',volume_shape/2)
print('find this=', midz - SC_start )
z_start = midz - 337/4
z_end = z_start + (volume_shape + 1) // 2

print('start, end', z_start, z_end)
print('volume shape and target length', volume_shape, z_end-z_start)

end - start= 184
half of volume= 168.5
find this= 111
start, end 153.75 322.75
volume shape and target length 337 169.0


In [9]:
%%time
#ng = NumpyToNeuroglancer(atlasV7_volume, [10000, 10000, 20000])
#ng.preview()
#ng.init_precomputed(OUTPUT_DIR)
#ng.add_segment_properties(get_segment_properties())
#ng.add_downsampled_volumes()
#ng.add_segmentation_mesh()

CPU times: user 8 µs, sys: 2 µs, total: 10 µs
Wall time: 17.2 µs
